## Optimization for nutrition

![title](Image/Image.jpg)

Hospital's management wishes to create a planned menu for lunch. This menu is devided into three great categories: **Vegetables**, **meats** and **desserts**. According with nutrition crew, one portion of each category must be served on the lunch, at least. The cust of each kind of food With your respective amount of **carbohydrates**, **vitamins**, **proteins** and **fats** is presented on the table **Tabela_alimentos.csv** below.

The minimal requirements of **carbohydrates**, **vitamins**, **proteins** and **fats** in a lunch are 5, 10, 10, 2 respectively. With these informations, we are going to build a balanced menu for a lunch, in order to reduce the hospital cost.

In this point the first step we need to do is to make an objective function for this problem. As we are interested in decreases hospital costs, our objective function is a function of each food cost times used portion of each food:

* $ f = \sum_{i=1}^n Cost*Food_i $

Next setp is to define constrained functions. This is builded with amount of carbohydrates, vitamins, proteins and fats in this specific food times portion of each food:

* $\begin{cases}  Carbohydrates = \sum_{i=1}^n x_{Carbohydrates}*Food_i \geq 5\\ 
Vitamins = \sum_{i=1}^n x_{Vitamins}*Food_i \geq 10\\
Proteins = \sum_{i=1}^n x_{Proteins}*Food_i \geq 10\\
Fats = \sum_{i=1}^n x_{Fats}*Food_i \geq 2\end{cases}$

Additional constraints must be added. Nutrition crew said all categories of foods must be used in the lunch menu. In a mathematical point this is:

* $\begin{cases} Vegetables = \sum_{i=1}^n Food_{Veg[i]} \geq 1\\
Meats = \sum_{i=1}^n Food_{Meats[i]} \geq 1\\
Desserts = \sum_{i=1}^n Food_{Des[i]} \geq 1\end{cases} $

**Once the equations are defined, is time to bring this to python!**

In [1]:
#-------------------------------------- Table importing ------------------------------#
import pandas as pd

#---------------------------------- Variables manipulation ---------------------------#
import numpy as np

#------------------------------------- Optimization tool -----------------------------#
from scipy.optimize import minimize

### Dataset information

In [2]:
df = pd.read_csv("Tabela_alimentos.csv", decimal=",")
df

,Alimentos,Carboidratos,Vitaminas,Proteínas,Gordura,"Custo por porção (1 unidade), $"
0,Ervilha,1,3,1,0,0.10
1,Feijão,1,5,2,0,0.12
2,Quiabo,1,5,1,0,0.13
3,Milho,2,6,1,2,0.09
4,Macarrão,4,2,1,1,0.10
5,Arroz,5,1,1,1,0.07
6,Frango,2,1,3,1,0.70
7,Bovina,3,8,5,2,1.20
8,Peixe,3,6,6,1,0.63
9,Laranja,1,3,1,1,0.28


### Objective function 

In [3]:
def func(x):
    
    f = 0
    custo = np.array(df["Custo por porção (1 unidade), $"])
    
    for i in range(len(x)):
        
        f += x[i] * custo[i]
    
    return f

### Constraints

In [4]:
#--------------------------------------- Carbohydrates constrain ------------------------------#
def const_carb(x, val = 5):
    
    carb = 0
        
    column = np.array(df["Carboidratos"])
        
    for j in range(len(x)):
            
        carb += x[j] * column[j]
        
    carb += -5
            
    return carb


#--------------------------------------- Vitamin constrain ------------------------------#
def const_vit(x, val = 10):
    
    vit = 0
        
    column = np.array(df["Vitaminas"])
        
    for j in range(len(x)):
            
        vit += x[j] * column[j]
        
    vit += -10
            
    return vit


#--------------------------------------- Proteins constrain ------------------------------#
def const_pro(x, val = 10):
    
    pro = 0
        
    column = np.array(df["Proteínas"])
        
    for j in range(len(x)):
            
        pro += x[j] * column[j]
        
    pro += -10
            
    return pro


#--------------------------------------- Fat constrain ------------------------------#
def const_fat(x, val = 2):
    
    fat = 0
        
    column = np.array(df["Gordura"])
        
    for j in range(len(x)):
            
        fat += x[j] * column[j]
        
    fat += -2
            
    return fat


#--------------------------------------- Vegetable constrain ------------------------------#
def const_veg(x):
    f = 0
    
    for i in range(0, 6):
        
        f += x[i]
        
    f += -1
    
    return f

#--------------------------------------- Meat constrain ------------------------------#
def const_meat(x):
    f = 0
    
    for i in range(6, 9):
        
        f += x[i]
        
    f += -1
    
    return f


#--------------------------------------- Dessert constrain ------------------------------#
def const_dess(x):
    f = 0
    
    for i in range(9, 13):
        
        f += x[i]
        
    f += -1
    
    return f

### Optimization

In [5]:
#----------------------------------------- Initial guess -------------------------------#
x0 = np.zeros(13)

#------------------------------------------ Constraints ----------------------------------#
const_c = {"type": 'ineq', 'fun': const_carb}
const_v = {"type": 'ineq', 'fun': const_vit}
const_p = {"type": 'ineq', 'fun': const_pro}
const_f = {"type": 'ineq', 'fun': const_fat}
const_ve = {"type": 'ineq', 'fun': const_veg}
const_me = {"type": 'ineq', 'fun': const_meat}
const_de = {"type": 'ineq', 'fun': const_dess}

const = [const_c, const_v, const_p, const_f, const_ve, const_me, const_de]

b = (0, 2)
bounds = (b,b,b,b,b,b,b,b,b,b,b,b,b)

#----------------------------------------- Optmization ---------------------------------#
solution = minimize(func, x0, method = 'SLSQP', bounds = bounds, constraints = const)

### Optimization results

In [6]:
solution

     fun: 0.9999999999999986
     jac: array([0.1       , 0.12000001, 0.13      , 0.09000001, 0.1       ,
       0.07      , 0.7       , 1.2       , 0.63      , 0.28000001,
       0.41999999, 0.15      , 0.12      ])
 message: 'Optimization terminated successfully.'
    nfev: 240
     nit: 16
    njev: 16
  status: 0
 success: True
       x: array([0. , 1.5, 0. , 0. , 0. , 1. , 0. , 0. , 1. , 0. , 0. , 0. , 1. ])

### Amount of each food

In [7]:
foods = np.array(df["Alimentos"])

for i, food in enumerate(foods):
    print("Portion(s) of ",food, " ----->", round(solution.x[i],2))

Portion(s) of  Ervilha  -----> 0.0
Portion(s) of  Feijão  -----> 1.5
Portion(s) of  Quiabo  -----> 0.0
Portion(s) of  Milho  -----> 0.0
Portion(s) of  Macarrão  -----> 0.0
Portion(s) of  Arroz  -----> 1.0
Portion(s) of  Frango  -----> 0.0
Portion(s) of  Bovina  -----> 0.0
Portion(s) of  Peixe  -----> 1.0
Portion(s) of  Laranja  -----> 0.0
Portion(s) of  Maçã  -----> 0.0
Portion(s) of  Pudim  -----> 0.0
Portion(s) of  Gelatina  -----> 1.0
